<a href="https://colab.research.google.com/github/dhsolutionspak/MSCUI/blob/main/Programming_for_AI_Assignment_Report%2027/12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
!pip install -U "tensorflow-text==2.13.*"

## Importing The Libaries

In [134]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import os
print(os.listdir("/content"))

['.config', 'logs', 'USvideos.csv', 'results', 'wandb', 'UScomments.csv', 'sample_data']


## Loading The Data

In [135]:
comments = pd.read_csv('/content/UScomments.csv', on_bad_lines='skip')
US_comments = comments.iloc[:1000]

In [136]:
US_comments.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


## Let's do some analysis and Data Cleaning on both the datasets.

In [137]:
US_comments.shape

(1000, 4)

In [138]:
US_comments.nunique()

,0
video_id,11
comment_text,991
likes,8
replies,5


In [139]:
US_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      1000 non-null   object
 1   comment_text  1000 non-null   object
 2   likes         1000 non-null   object
 3   replies       1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [62]:
US_comments.isnull().sum()

,0
video_id,0
comment_text,0
likes,0
replies,0


In [140]:
US_comments.likes = US_comments.likes.astype(int)
US_comments.replies = US_comments.replies.astype(int)

In [141]:
US_comments.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [163]:
import re
from transformers import BertTokenizer

# Assuming US_comments is your DataFrame
# Display the first few rows of the dataset
print(US_comments.head())

# Data Cleaning Function
def clean_text(text):
    """Clean the text by removing special characters, emojis, and extra spaces."""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply cleaning to the comment_text column
US_comments['cleaned_text'] = US_comments['comment_text'].apply(clean_text)

# Initialize BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the cleaned text
def tokenize_text(text):
    """Tokenize the cleaned text using BERT tokenizer."""
    return tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")

US_comments['tokenized'] = US_comments['cleaned_text'].apply(tokenize_text)

# Display the cleaned and tokenized data
print(US_comments[['comment_text', 'cleaned_text', 'tokenized']].head())


      video_id                                       comment_text  likes  \
0  XpVt6Z1Gjjo        l o g a n   p a u l   i t ' s   ‼ ️ ‼ ️ ‼ ️      4   
1  XpVt6Z1Gjjo  i ' v e   b e e n   f o l l o w i n g   f r o ...      3   
2  XpVt6Z1Gjjo                          k o n g   m a v e r i c k      3   
3  XpVt6Z1Gjjo                                a t t e n d a n c e      3   
4  XpVt6Z1Gjjo                                    t r e n d i n g      3   

   replies  Sentiment Scores Sentiment  
0        0               0.0   Neutral  
1        0               0.0   Neutral  
2        0               0.0   Neutral  
3        0               0.0   Neutral  
4        0               0.0   Neutral  
                                        comment_text  \
0        l o g a n   p a u l   i t ' s   ‼ ️ ‼ ️ ‼ ️   
1  i ' v e   b e e n   f o l l o w i n g   f r o ...   
2                          k o n g   m a v e r i c k   
3                                a t t e n d a n c e   
4                

## Removing Punctuations, Numbers and Special Characters.

In [142]:
US_comments['comment_text'] = US_comments['comment_text'].str.replace("[^a-zA-Z#]", " ")

## Removing Short Words.

In [143]:
US_comments['comment_text'] = US_comments['comment_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

## Changing the text to lower case.

In [144]:
US_comments['comment_text'] = US_comments['comment_text'].apply(lambda x:x.lower())

## Tokenization

In [145]:
tokenized_tweet = US_comments['comment_text'].apply(lambda x: x.split())


## Lemmatization

In [146]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [147]:
wnl = WordNetLemmatizer()

In [148]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
tokenized_tweet.apply(lambda x: [wnl.lemmatize(i) for i in x if i not in set(stopwords.words('english'))])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,comment_text
0,"[logan, paul, ‼️‼️‼️]"
1,"[i've, following, start, vine, channel, seen, ..."
2,"[kong, maverick]"
3,[attendance]
4,[trending]
...,...
995,"[generic, attention, comment]"
996,"[zero, amount, funny]"
997,"[honest?, random, color, hairgender, study, le..."
998,"[what,, joke, can't, apply, going, call, woman..."


In [158]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

In [159]:
US_comments['comment_text'] = tokenized_tweet

In [160]:
US_comments.head()

,video_id,comment_text,likes,replies,Sentiment Scores,Sentiment
0,XpVt6Z1Gjjo,l o g a n p a u l i t ' s ‼ ️ ‼ ️ ‼ ️,4,0,0.0,Neutral
1,XpVt6Z1Gjjo,i ' v e b e e n f o l l o w i n g f r o ...,3,0,0.0,Neutral
2,XpVt6Z1Gjjo,k o n g m a v e r i c k,3,0,0.0,Neutral
3,XpVt6Z1Gjjo,a t t e n d a n c e,3,0,0.0,Neutral
4,XpVt6Z1Gjjo,t r e n d i n g,3,0,0.0,Neutral


## Let's do the Sentiment Analysis on the US Comments Dataset

In [151]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [152]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

## Setting The Sentiment Scores

In [153]:
US_comments['Sentiment Scores'] = US_comments['comment_text'].apply(lambda x:sia.polarity_scores(x)['compound'])

In [154]:
US_comments.head()

,video_id,comment_text,likes,replies,Sentiment Scores
0,XpVt6Z1Gjjo,logan paul it's ‼️‼️‼️,4,0,0.0
1,XpVt6Z1Gjjo,i've been following from start your vine chann...,3,0,0.0
2,XpVt6Z1Gjjo,kong maverick,3,0,0.0
3,XpVt6Z1Gjjo,attendance,3,0,0.0
4,XpVt6Z1Gjjo,trending,3,0,0.0


## Classifying the Sentiment scores as Positive, Negative and Neutral

In [155]:
US_comments['Sentiment'] = US_comments['Sentiment Scores'].apply(lambda s : 'Positive' if s > 0 else ('Neutral' if s == 0 else 'Negative'))

In [123]:
US_comments.head()

,video_id,comment_text,likes,replies,Sentiment Scores,Sentiment
0,XpVt6Z1Gjjo,logan paul it's ‼️‼️‼️,4,0,0.0,Neutral
1,XpVt6Z1Gjjo,i've been following from start your vine chann...,3,0,0.0,Neutral
2,XpVt6Z1Gjjo,kong maverick,3,0,0.0,Neutral
3,XpVt6Z1Gjjo,attendance,3,0,0.0,Neutral
4,XpVt6Z1Gjjo,trending,3,0,0.0,Neutral


In [156]:
US_comments.Sentiment.value_counts()

,count
Sentiment,
Positive,421
Neutral,392
Negative,187


In [ ]:
!pip install tensorflow-text

In [165]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [166]:
US_comments['Sentiment'] = US_comments['Sentiment Scores'].apply(lambda s : 'Positive' if s > 0 else ('Neutral' if s == 0 else 'Negative'))
# X and y are derived from the DataFrame
X = US_comments['comment_text']
y = US_comments['Sentiment']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [167]:
X_train

,comment_text
29,l o v e m u c h l o v e m e e t l i v ...
535,t r u m p l i t e r a l l y d i d n ' t ...
695,a m a z i n g !
557,0 7 : 3 2 m a k e g r e a t a g a i n ...
836,h o l d f u v k n e t f l i x i n c l u ...
...,...
106,"d o n t s o r r y p e w d s , e v e r y ..."
270,a b s o l u t e l y b e a u t i f u l a p ...
860,s h o u l d m o v i e n e x t ! !
435,. . . \ n \ n . . w h e r e p a r k f l y ...


In [168]:
X_test

,comment_text
521,s e t h f o r g o t r a m p a n t n e p ...
737,d i d n ' t c o u n t r y ! ! !
740,g o i n g t h o s e l o o k r i d i c u ...
660,t h e y h a v e s o m e s o r t t o u ...
411,‪ s u b s c r i b e p l e a s e ! h t t p ...
...,...
408,h o l y s h i t f r a n c o n a i l s
332,w a t c h c l i c k i n g h e r e p e o ...
208,l o o k s o u t d a t e d w i t h c h r ...
613,b a t t e r y i c o n a l r e a d y c h ...


In [191]:
import tensorflow as tf
import tensorflow_hub as hub

# Define the preprocessor and encoder layers
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3",
    name="bert_preprocessor",
)
encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4",
    trainable=True,
    name="bert_encoder",
)

# Define the model
def build_bert_model():
    # Input layer
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text_input")

    # Preprocess the input text
    encoder_inputs = preprocessor(text_input)

    # Pass the preprocessed text to the BERT encoder
    outputs = encoder(encoder_inputs)

    # Extract the pooled output
    pooled_output = outputs["pooled_output"]

    # Add a dropout layer
    dropout = tf.keras.layers.Dropout(0.3, name="dropout")(pooled_output)

    # Add a dense layer for classification
    classifier = tf.keras.layers.Dense(3, activation="softmax", name="classifier")(dropout)

    # Define the model
    model = tf.keras.Model(inputs=text_input, outputs=classifier, name="BERT_Classifier")

    return model

# Build the model
model = build_bert_model()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Print the model summary
model.summary()

# Assume you have X_train, y_train, X_test, y_test ready
# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=3,
    batch_size=16,
    validation_data=(X_test, y_test),
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Save the fine-tuned model
model.save("fine_tuned_bert_keras")


ValueError: Exception encountered when calling layer 'bert_preprocessor' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'bert_preprocessor' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=False, name=text_input>
  • training=None

In [170]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4", trainable=True)


In [187]:
# Define the model function
def bert_model(text_input):
    encoder_inputs = preprocessor(text_input)
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]
    drop_out = tf.keras.layers.Dropout(0.3, name='dropout')(pooled_output)
    output = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(drop_out)
    return output


In [188]:
# Input layer
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

In [189]:
# Apply the model function to the input
output = bert_model(text_input)


ValueError: Exception encountered when calling layer 'keras_layer_48' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_48' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=string, sparse=False, name=text>
  • training=None

In [ ]:
# Build the model
model = tf.keras.Model(inputs=[text_input], outputs=[output])

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train,
                    y_train,
                    epochs=3,
                    batch_size=16,
                    validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# Save the fine-tuned model
model.save('fine_tuned_bert_keras')